# Analyze Price Data

In [1]:
from datetime import datetime
import os
import pandas as pd
import numpy as np

# Get Dates

In [2]:
def get_dates(root_folder):
    files = os.listdir(root_folder)
    return sorted([datetime.strptime(file.split("_")[1].split(".")[0], '%d-%m-%Y') 
            for file in files if ".csv" in file])

In [3]:
root_folder = "raw_data"
dates = get_dates(root_folder)

# Get Data

In [4]:
def get_data(dates):
    data_list = [pd.read_csv("raw_data/data_{0}.csv".format(date.strftime('%d-%m-%Y'))) for date in dates]
    return pd.concat(data_list)

In [5]:
final_data = get_data(dates)

In [6]:
final_data.head()

,Producto,Categoría,Anterior,Último,Diferencia,Medida,Tendencia,Fecha
0,Cereales,Trigo panificable,"0,00 / 0,00","147,00","147,00",Eur / Tm.,NaN,02-01-2006
1,Cereales,Cebada,"0,00 / 0,00","145,00","145,00",Eur / Tm.,NaN,02-01-2006
2,Cereales,Avena,"0,00 / 0,00","145,00","145,00",Eur / Tm.,NaN,02-01-2006
3,Cereales,Centeno,"0,00 / 0,00","139,00","139,00",Eur / Tm.,NaN,02-01-2006
4,Cereales,Maíz,"0,00 / 0,00","142,00","142,00",Eur / Tm.,NaN,02-01-2006


In [7]:
final_data[final_data["Categoría"]=="Cebada"].head()

,Producto,Categoría,Anterior,Último,Diferencia,Medida,Tendencia,Fecha
1,Cereales,Cebada,"0,00 / 0,00","145,00","145,00",Eur / Tm.,NaN,02-01-2006
1,Cereales,Cebada,"145,00","145,00","0,00",Eur / Tm.,NaN,09-01-2006
1,Cereales,Cebada,"145,00","145,00","0,00",Eur / Tm.,NaN,16-01-2006
1,Cereales,Cebada,"145,00","145,00","0,00",Eur / Tm.,NaN,23-01-2006
1,Cereales,Cebada,"145,00","144,00","-1,00",Eur / Tm.,NaN,30-01-2006


In [8]:
final_data[final_data["Categoría"]=="Cebada"].tail()

,Producto,Categoría,Anterior,Último,Diferencia,Medida,Tendencia,Fecha
1,Cereales,Cebada,"154,00 / 0,00","155,00 / 0,00","1,00",Eur / Tm.,NaN,23-07-2018
1,Cereales,Cebada,"155,00 / 0,00","159,00 / 0,00","4,00",Eur / Tm.,NaN,30-07-2018
1,Cereales,Cebada,"159,00 / 0,00","165,00 / 0,00","6,00",Eur / Tm.,NaN,06-08-2018
1,Cereales,Cebada,"165,00 / 0,00","175,00 / 0,00","10,00",Eur / Tm.,NaN,13-08-2018
1,Cereales,Cebada,"165,00 / 0,00","175,00 / 0,00","10,00",Eur / Tm.,NaN,13-08-2018


# Clean Data

In [9]:
def get_price(row, row_name):
    value = float(row[row_name].split("/")[0].strip().replace(",", "."))
    if value == 0.0:
        return np.nan
    else:
        return value

In [10]:
final_data["Anterior"] = final_data.apply(lambda row: get_price(row, "Anterior"), axis=1)

In [11]:
final_data["Último"] = final_data.apply(lambda row: get_price(row, "Último"), axis=1)

In [12]:
final_data["Diferencia"] = final_data.apply(lambda row: get_price(row, "Diferencia"), axis=1)

In [13]:
final_data[final_data["Categoría"]=="Cebada"].tail()

,Producto,Categoría,Anterior,Último,Diferencia,Medida,Tendencia,Fecha
1,Cereales,Cebada,154.0,155.0,1.0,Eur / Tm.,NaN,23-07-2018
1,Cereales,Cebada,155.0,159.0,4.0,Eur / Tm.,NaN,30-07-2018
1,Cereales,Cebada,159.0,165.0,6.0,Eur / Tm.,NaN,06-08-2018
1,Cereales,Cebada,165.0,175.0,10.0,Eur / Tm.,NaN,13-08-2018
1,Cereales,Cebada,165.0,175.0,10.0,Eur / Tm.,NaN,13-08-2018


# Get Grouped Data

In [14]:
dict_df = {key:df for key, df in final_data.groupby("Categoría")}

# Impute Missing Data

In [15]:
for key in dict_df.keys():
    dict_df[key]["Anterior"] = dict_df[key]["Anterior"].interpolate(axis=0)
    dict_df[key]["Último"] = dict_df[key]["Último"].interpolate(axis=0)

# Export Weekly Data

In [16]:
writer_weekly = pd.ExcelWriter('weekly_data.xlsx', engine='xlsxwriter')

In [17]:
for key in dict_df.keys():
    sheet_name = key[:30]
    chars = '[]:*?/\"'
    for c in chars:
        sheet_name = sheet_name.replace(c, "")
    print(sheet_name)
    dict_df[key].to_excel(writer_weekly, sheet_name=sheet_name)

Especial menor 270 Kg
Vida
Primera mayor  270 Kg
Recorte Graso
 4 A 6 Kg
Extra hembra
Presa
D. O. Guijuelo
Tercera
Segunda
Macho
Segunda mayor 350 Kg
Magro de Bellota
Bellota
Primales ibérico cruzado
Bellota 160-170 Kg
Lechones Hasta 23 Kg
Marranos ibéricos cruzados
Especial macho
Primera macho
15,01 a 19 KG.
13,01 a 15 KG.
Primera menor 270 Kg
Primera hembra
Lechones hasta 23 kg (Recogida
Primera hembra   de 180 a 220 
Paja paquete pequeño
Garbanzo pedrosillano
Girasol A.O.
Especial menor 350 Kg
Extra mayor 270 Kg
Secreto
Especial mayor 350 Kg
Bellota 75 % IB
Normal
Panceta a cuadro
Primales
Barriguera
Pluma de lomo
Avena
Lomo Cebo de Campo
Grasas Industriales
25,5 a 28 KG.
Fabula, Marfona
Marranos 30 - 60 Kg
Segunda macho
Tostones 5 - 7 Kg
Tapilla
11,01 a 13 KG.
Agria
Veza henificada
Primera
Spunta
Reproductoras
Fábrica
Tocino de Lomo
segunda mayor  270 Kg
Marranos ibéricos puros
Trigo panificable
Lenteja armuña D.O.
Reproductoras < de 6 meses
Carrillera
Segunda menor 350 Kg
Bellota 

In [18]:
writer_weekly.save()

# Group & Export Monthly Data

In [84]:
dict_df["Cebada"].head()

,Producto,Categoría,Anterior,Último,Diferencia,Medida,Tendencia,Fecha,Fecha-Sort
1,Cereales,Cebada,NaN,145.0,145.0,Eur / Tm.,NaN,02-01-2006,2006-01-01
1,Cereales,Cebada,145.0,145.0,NaN,Eur / Tm.,NaN,09-01-2006,2006-01-01
1,Cereales,Cebada,145.0,145.0,NaN,Eur / Tm.,NaN,16-01-2006,2006-01-01
1,Cereales,Cebada,145.0,145.0,NaN,Eur / Tm.,NaN,23-01-2006,2006-01-01
1,Cereales,Cebada,145.0,144.0,-1.0,Eur / Tm.,NaN,30-01-2006,2006-01-01


In [85]:
def get_year_month(row):
    date = datetime.strptime(row["Fecha"], '%d-%m-%Y')
    return date.replace(day=1)

In [86]:
def format_year_month(row):
    return row.strftime('%m-%Y')

In [87]:
writer_monthly = pd.ExcelWriter('monthly_data.xlsx', engine='xlsxwriter')

In [88]:
dict_df_montly = {}
for key in dict_df.keys():
    sheet_name = key[:30]
    chars = '[]:*?/\"'
    for c in chars:
        sheet_name = sheet_name.replace(c, "")
    print(sheet_name)
    month_data = dict_df[key]
    month_data["Fecha-Sort"] = month_data.apply(lambda row: get_year_month(row), axis=1)
    grouped_data = month_data.groupby(["Producto", "Categoría", "Fecha-Sort", "Medida"])["Último"].mean().reset_index()
    grouped_data["Fecha"] = grouped_data["Fecha-Sort"].apply(lambda row: format_year_month(row))
    grouped_data = grouped_data.drop(["Fecha-Sort"], axis=1)
    grouped_data.to_excel(writer_monthly, sheet_name=sheet_name)

Especial menor 270 Kg
Vida
Primera mayor  270 Kg
Recorte Graso
 4 A 6 Kg
Extra hembra
Presa
D. O. Guijuelo
Tercera
Segunda
Macho
Segunda mayor 350 Kg
Magro de Bellota
Bellota
Primales ibérico cruzado
Bellota 160-170 Kg
Lechones Hasta 23 Kg
Marranos ibéricos cruzados
Especial macho
Primera macho
15,01 a 19 KG.
13,01 a 15 KG.
Primera menor 270 Kg
Primera hembra
Lechones hasta 23 kg (Recogida
Primera hembra   de 180 a 220 
Paja paquete pequeño
Garbanzo pedrosillano
Girasol A.O.
Especial menor 350 Kg
Extra mayor 270 Kg
Secreto
Especial mayor 350 Kg
Bellota 75 % IB
Normal
Panceta a cuadro
Primales
Barriguera
Pluma de lomo
Avena
Lomo Cebo de Campo
Grasas Industriales
25,5 a 28 KG.
Fabula, Marfona
Marranos 30 - 60 Kg
Segunda macho
Tostones 5 - 7 Kg
Tapilla
11,01 a 13 KG.
Agria
Veza henificada
Primera
Spunta
Reproductoras
Fábrica
Tocino de Lomo
segunda mayor  270 Kg
Marranos ibéricos puros
Trigo panificable
Lenteja armuña D.O.
Reproductoras < de 6 meses
Carrillera
Segunda menor 350 Kg
Bellota 

In [89]:
writer_monthly.save()